In [80]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from gurobipy import *
import pickle
from itertools import chain, combinations

In [81]:
df = pd.read_csv("data.csv", sep=";", index_col=0)

In [82]:
int_col =[str(i) for i in range(1,21)]

In [83]:
class Project:
    def __init__(self, num=0, desc=0, a1=0, a2=0, w=[1/2, 1/2], cost=1):
        self.num = num
        self.desc = desc
        self.a1 = a1
        self.a2 = a2    
        self.cost = cost
        self.w = w
                
    def cal_value(self):
        self.value = (self.a1/4)*self.w[0] + (self.a2/4)*self.w[1]
        
    def describe(self):
        print("Project " + self.desc)
        print("a1: ", self.a1)
        print("a2: ", self.a2)
        print("cost: ", self.cost)
        print("w: ", self.w)
        print("value: ", self.value)
        

In [84]:
project_list = []
for i in range(20):
    r = df.iloc[i]
    
    p = Project(num=i+1)
    p.desc = r["Action idea name"]
    p.a1 = float(r["Hiilineutraalisuus carbon neutrality (avg)  "].replace(',', '.'))
    p.a2 = float(r["Varautuminen Resilience (avg)  "].replace(',', '.'))
    p.pos_impact = float(r["Positiiviset vaikutukset (avg)  "].replace(',', '.'))
    p.neg_impact = float(r["Negatiiviset vaikutukset (neg. impact)  (avg)  "].replace(',', '.'))
    p.cal_value()
    
    project_list.append(p)

In [85]:
df2 = pd.read_excel("inter_updated.xlsx", header=None)

In [86]:
score_list = []
for i in range(20):
    print("i: ", i)
    for j in range(i, 19):
        print("j: ",j)
        score = df2.iloc[i,j]
        if type(score) == str:
            score_list.append(score.replace(' ', ''))

i:  0
j:  0
j:  1
j:  2
j:  3
j:  4
j:  5
j:  6
j:  7
j:  8
j:  9
j:  10
j:  11
j:  12
j:  13
j:  14
j:  15
j:  16
j:  17
j:  18
i:  1
j:  1
j:  2
j:  3
j:  4
j:  5
j:  6
j:  7
j:  8
j:  9
j:  10
j:  11
j:  12
j:  13
j:  14
j:  15
j:  16
j:  17
j:  18
i:  2
j:  2
j:  3
j:  4
j:  5
j:  6
j:  7
j:  8
j:  9
j:  10
j:  11
j:  12
j:  13
j:  14
j:  15
j:  16
j:  17
j:  18
i:  3
j:  3
j:  4
j:  5
j:  6
j:  7
j:  8
j:  9
j:  10
j:  11
j:  12
j:  13
j:  14
j:  15
j:  16
j:  17
j:  18
i:  4
j:  4
j:  5
j:  6
j:  7
j:  8
j:  9
j:  10
j:  11
j:  12
j:  13
j:  14
j:  15
j:  16
j:  17
j:  18
i:  5
j:  5
j:  6
j:  7
j:  8
j:  9
j:  10
j:  11
j:  12
j:  13
j:  14
j:  15
j:  16
j:  17
j:  18
i:  6
j:  6
j:  7
j:  8
j:  9
j:  10
j:  11
j:  12
j:  13
j:  14
j:  15
j:  16
j:  17
j:  18
i:  7
j:  7
j:  8
j:  9
j:  10
j:  11
j:  12
j:  13
j:  14
j:  15
j:  16
j:  17
j:  18
i:  8
j:  8
j:  9
j:  10
j:  11
j:  12
j:  13
j:  14
j:  15
j:  16
j:  17
j:  18
i:  9
j:  9
j:  10
j:  11
j:  12
j:  13
j:  14
j:  15
j

In [87]:
score_dict = {'+':0.33, '++': 0.66, '+++': 1, '+++(+)': (1+1.33)/2,'+/-':0, '-':-0.33, '0/+':(0.33/2) }

In [145]:
score_dict.keys()

dict_keys(['+', '++', '+++', '+++(+)', '+/-', '-', '0/+'])

In [146]:
score_dict.values()

dict_values([0.33, 0.66, 1, 1.165, 0, -0.33, 0.165])

In [147]:
score_df = pd.DataFrame()

In [151]:
score_df["effect"] = score_dict.keys()
score_df["gamma"] = score_dict.values()
score_df.to_csv("effect")

In [179]:
int_df = pd.DataFrame(np.nan, index = list(range(0,20)), columns = list(range(0,20))) 
for i in range(len(inter_subset)):
    int_df.iloc[inter_subset[i][0], inter_subset[i][1]] = inter_effect[i]
int_df.to_csv("inter_effect.csv", sep = ";", decimal="," )

In [112]:
inter_subset = []
inter_effect = []

for i in range(20):
    for j in range(i, 19):
        score = df2.iloc[i,j]
        if type(score) == str:
            score = score.replace(' ', '')
            
            try:
                inter_effect.append(score_dict[score])
                inter_subset.append([i,j+1])
            except:
                pass

In [113]:
for i in range(len(inter_subset)):
    print(inter_subset[i])
    print(inter_effect[i])
    print()


[0, 3]
0.165

[0, 5]
0.165

[0, 11]
0.165

[1, 2]
0.66

[1, 3]
0.33

[1, 4]
0.66

[1, 5]
0

[1, 9]
0.66

[1, 11]
0.66

[1, 12]
1.165

[1, 14]
1

[1, 16]
0.66

[2, 3]
0.33

[2, 4]
0.33

[2, 6]
0.33

[2, 8]
0.33

[2, 19]
0.33

[3, 6]
0.33

[3, 8]
0.33

[3, 17]
0.33

[4, 12]
1

[4, 14]
1

[5, 12]
0.33

[5, 14]
0.66

[6, 7]
0.33

[6, 8]
1

[6, 9]
0.33

[6, 10]
0.33

[6, 12]
0.66

[6, 14]
0.66

[6, 16]
0.33

[7, 8]
1

[7, 9]
1

[7, 12]
0.33

[7, 14]
0.33

[8, 11]
0.33

[8, 14]
0.66

[8, 15]
-0.33

[8, 16]
0.33

[8, 17]
0.33

[8, 19]
0.33

[9, 11]
0.33

[9, 12]
0.33

[9, 14]
0.66

[9, 16]
0.33

[9, 17]
0.33

[9, 19]
0.33

[10, 11]
0.33

[10, 12]
0.33

[10, 13]
1

[10, 14]
0.33

[10, 15]
0.66

[10, 16]
0.33

[10, 17]
0.33

[10, 18]
0.33

[10, 19]
0.33

[11, 12]
1

[11, 13]
1

[11, 14]
0.66

[11, 15]
0.33

[11, 16]
0.33

[11, 17]
0.33

[11, 18]
0.33

[11, 19]
0.66

[12, 13]
1

[12, 14]
1

[12, 15]
1

[12, 18]
0.33

[12, 19]
0.66

[13, 14]
0.33

[13, 15]
1

[13, 19]
0.33

[14, 15]
1

[14, 16]
0

In [129]:
class SolveProblem:
    def __init__(self, budget_const_pct, inter_subset, inter_effect, project_list):
        self.budget_const_pct = budget_const_pct
        
        self.m = len(project_list)
        self.project_values = [x.value for x in project_list]
        self.project_costs = [x.cost for x in project_list] 
        
        self.optimal_portfolios = []
        self.optimal_values = []
        
        self.interaction_effects_dict = {}
        
        self.interaction_subsets = inter_subset
        self.inter_gamma = inter_effect
        
    def run(self):
        
        #A model that generates the first optimal solution
        model = Model("Portfolio opimization - Additive")   

        #Indexes
        projects = range(len(project_list))

        z = []

        for i in projects:
            z.append(model.addVar(vtype=GRB.BINARY, name="z_%d" % (i)))
        model.update()

        #budget constraints
        self.budget = self.budget_const_pct * sum(self.project_costs)

        model.addConstr(quicksum(z[j]*self.project_costs[j] for j in projects) <= self.budget, "Budget Constraint")
        model.update()

        #objective function
        model.setObjective(quicksum(z[j]*self.project_values[j] for j in projects))
        model.modelSense = GRB.MAXIMIZE
        model.update()

        #optimize the model
        model.optimize()

        #Collect optimal decision variables

        sol_dict = {}
        sol_list = model.getVars()
        
        for i in range(self.m):
            sol_dict[sol_list[i].VarName] = sol_list[i].X
        
        self.optimal_portfolios.append(sol_dict)
        self.optimal_values.append(model.objVal)
        
        for beta in list(np.arange(0, 0.1, 0.01)) + list(np.arange(0.1, 1, 0.1))+list(np.arange(1,10,1)):
            
            self.interaction_effects = []

            for gamma in self.inter_gamma:
                print("gamma", gamma)
                S_k = gamma*0.73*beta
                print("S_k", S_k)
                self.interaction_effects.append(S_k)
                
            output = np.zeros((len(self.interaction_subsets), 20))
            for i, ix in enumerate(self.interaction_subsets):
                output[i][ix] = 1
            self.interaction_subsets_binary = output.T
            self.interaction_subsets_size = output.sum(axis=1)
            
            self.interaction_effects_dict[beta] = self.interaction_effects
            
                                    
            model = Model("Portfolio optimization - with interaction")

            J = self.m #rename the number of project for the simplicity
            
            #select interaction subsets that should be included in the model
            interaction_subsets = self.interaction_subsets
            interaction_subsets_binary = self.interaction_subsets_binary
            interaction_subsets_size = self.interaction_subsets_size
            interaction_effects= self.interaction_effects
    
            
            #Indexes for the model
            projects = range(J)
            interactions = range(len(self.interaction_subsets))

            #Decision variables z_j
            z = []
            for i in projects:
                z.append(model.addVar(vtype=GRB.BINARY, name="z_%d" % (i)))
            model.update()

            #Decision variable g_k
            g = []
            for k in interactions:
                g.append(model.addVar(vtype=GRB.BINARY, name="g_%d" % (k)))
            model.update()

            #budget constraints
            self.budget = self.budget_const_pct * sum(self.project_costs)

            model.addConstr(quicksum(z[j]*self.project_costs[j] for j in projects) <= self.budget, "Budget Constraint")
            model.update()

            #interaction constraints left hand side
            for k in interactions:
                model.addConstr(quicksum(z[j]*interaction_subsets_binary[j][k] for j in projects) -
                               interaction_subsets_size[k] + 1 <= J*g[k])
            model.update()
            
            #interaction constraints right hand side
            for k in interactions:
                model.addConstr(J*g[k] <= quicksum(z[j]*interaction_subsets_binary[j][k] for j in projects) -
                               interaction_subsets_size[k] + J)
            model.update()
            
            #objective function
            model.setObjective(quicksum(z[j]*self.project_values[j] for j in projects) + quicksum(g[k]*interaction_effects[k] for k in interactions))

            model.modelSense = GRB.MAXIMIZE
            model.update()

            #optimize the model
            model.optimize()

            sol_dict = {}
            sol_list = model.getVars()
            for i in range(J + len(self.interaction_subsets)):
                sol_dict[sol_list[i].VarName] = sol_list[i].X
                
            self.optimal_portfolios.append(sol_dict)
            self.optimal_values.append(model.objVal)
            print("Beta: ", beta)
        
        
        return self.optimal_portfolios

In [130]:
def make_project(w):
    project_list = []
    for i in range(20):
        r = df.iloc[i]

        p = Project(num=i+1)
        p.desc = r["Action idea name"]
        p.a1 = float(r["Hiilineutraalisuus carbon neutrality (avg)  "].replace(',', '.'))
        p.a2 = float(r["Varautuminen Resilience (avg)  "].replace(',', '.'))
        p.pos_impact = float(r["Positiiviset vaikutukset (avg)  "].replace(',', '.'))
        p.neg_impact = float(r["Negatiiviset vaikutukset (neg. impact)  (avg)  "].replace(',', '.'))
        p.w = w
        p.cal_value()

        project_list.append(p)
    return project_list

In [131]:
p1 = make_project([0.5, 0.5])
p2 = make_project([0.67, 0.33])
p3 = make_project([1,0])

In [132]:
d = {"[0.5, 0.5]":[p.value for p in p1], "[0.67, 0.33]":[p.value for p in p2], "[1,0]":[p.value for p in p3]}

In [133]:
pd.DataFrame(d).to_csv("utilities.csv", sep = ";", decimal=",")

In [134]:
s1 = SolveProblem(budget_const_pct=0.5, inter_subset=inter_subset, inter_effect=inter_effect, project_list=p1)
s1.run()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1 rows, 20 columns and 20 nonzeros
Model fingerprint: 0x37cc612d
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 1e+01]
Found heuristic solution: objective 7.8325000
Presolve removed 1 rows and 20 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 4 available processors)

Solution count 2: 8.33375 7.8325 

Optimal solution found (tolerance 1.00e-04)
Best objective 8.333750000000e+00, best bound 8.333750000000e+00, gap 0.0000%
gamma 0.165
S_k 0.0
gamma 0.165
S_k 0.0
gamma 0.165
S_k 0.0
gamma 0.66
S_k 0.0
gamma 0.33
S_k 0.0
gamma 0.66
S_k 0.0
gamma 0
S_k 0.0
gamma 

S_k 0.0146
gamma 1
S_k 0.0146
gamma 0.33
S_k 0.004818
gamma 0.66
S_k 0.009636
gamma 0.33
S_k 0.004818
gamma 1
S_k 0.0146
gamma 0.33
S_k 0.004818
gamma 1
S_k 0.0146
gamma 0.33
S_k 0.004818
gamma 0.33
S_k 0.004818
gamma 0.66
S_k 0.009636
gamma -0.33
S_k -0.004818
gamma 0.66
S_k 0.009636
gamma 0.33
S_k 0.004818
gamma 0.33
S_k 0.004818
gamma 1
S_k 0.0146
gamma 0.33
S_k 0.004818
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 165 rows, 102 columns and 512 nonzeros
Model fingerprint: 0x9900b165
Variable types: 0 continuous, 102 integer (102 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [2e-03, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 3.6662500
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 163 rows, 102 columns, 506 nonzeros
Variable types: 0 continuous,


    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    9.05301    0   34    8.12795    9.05301  11.4%     -    0s
H    0     0                       8.8088200    9.05301  2.77%     -    0s
     0     0     cutoff    0         8.80882    8.80882  0.00%     -    0s

Cutting planes:
  Gomory: 3
  Cover: 24
  RLT: 13

Explored 1 nodes (65 simplex iterations) in 0.06 seconds (0.00 work units)
Thread count was 4 (of 4 available processors)

Solution count 3: 8.80882 8.12795 3.66625 

Optimal solution found (tolerance 1.00e-04)
Best objective 8.808820000000e+00, best bound 8.808820000000e+00, gap 0.0000%
Beta:  0.04
gamma 0.165
S_k 0.0060225
gamma 0.165
S_k 0.0060225
gamma 0.165
S_k 0.0060225
gamma 0.66
S_k 0.02409
gamma 0.33
S_k 0.012045
gamma 0.66
S_k 0.02409
gamma 0
S_k 0.0
gamma 0.66
S_k 0.02409
gamma 0.66
S_k 0.02409
gamma 1.165
S_k 0.042522500000000005
gamma 1
S_k 0.0365
gamm

S_k 0.033726000000000006
gamma 0.66
S_k 0.033726000000000006
gamma 0.33
S_k 0.016863000000000003
gamma 1
S_k 0.051100000000000007
gamma 1
S_k 0.051100000000000007
gamma 0.33
S_k 0.016863000000000003
gamma 0.33
S_k 0.016863000000000003
gamma 0.33
S_k 0.016863000000000003
gamma 0.66
S_k 0.033726000000000006
gamma -0.33
S_k -0.016863000000000003
gamma 0.33
S_k 0.016863000000000003
gamma 0.33
S_k 0.016863000000000003
gamma 0.33
S_k 0.016863000000000003
gamma 0.33
S_k 0.016863000000000003
gamma 0.33
S_k 0.016863000000000003
gamma 0.66
S_k 0.033726000000000006
gamma 0.33
S_k 0.016863000000000003
gamma 0.33
S_k 0.016863000000000003
gamma 0.33
S_k 0.016863000000000003
gamma 0.33
S_k 0.016863000000000003
gamma 0.33
S_k 0.016863000000000003
gamma 1
S_k 0.051100000000000007
gamma 0.33
S_k 0.016863000000000003
gamma 0.66
S_k 0.033726000000000006
gamma 0.33
S_k 0.016863000000000003
gamma 0.33
S_k 0.016863000000000003
gamma 0.33
S_k 0.016863000000000003
gamma 0.33
S_k 0.016863000000000003
gamma 1
S_

Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 165 rows, 102 columns and 512 nonzeros
Model fingerprint: 0x1d8d37e2
Variable types: 0 continuous, 102 integer (102 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e-02, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 3.6662500
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 163 rows, 102 columns, 506 nonzeros
Variable types: 0 continuous, 102 integer (102 binary)
Found heuristic solution: objective 8.4425840

Root relaxation: objective 1.003031e+01, 25 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   10.03031    0   39    8.44258   10.03031  18.8%     -    0s
H    0     0                       9.4158465   10.

...

S_k 1.46
gamma 0.33
S_k 0.4818
gamma 0.66
S_k 0.9636
gamma 0.33
S_k 0.4818
gamma 1
S_k 1.46
gamma 0.33
S_k 0.4818
gamma 1
S_k 1.46
gamma 0.33
S_k 0.4818
gamma 0.33
S_k 0.4818
gamma 0.66
S_k 0.9636
gamma -0.33
S_k -0.4818
gamma 0.66
S_k 0.9636
gamma 0.33
S_k 0.4818
gamma 0.33
S_k 0.4818
gamma 1
S_k 1.46
gamma 0.33
S_k 0.4818
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 165 rows, 102 columns and 512 nonzeros
Model fingerprint: 0xba38eda7
Variable types: 0 continuous, 102 integer (102 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [2e-01, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 3.6662500
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 163 rows, 102 columns, 506 nonzeros
Variable types: 0 continuous, 102 integer (102 binary)
Found heuristic solution:

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 165 rows, 102 columns and 512 nonzeros
Model fingerprint: 0x0e647ca3
Variable types: 0 continuous, 102 integer (102 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [5e-01, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 3.6662500
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 163 rows, 102 columns, 506 nonzeros
Variable types: 0 continuous, 102 integer (102 binary)
Found heuristic solution: objective 38.9812000

Root relaxation: objective 9.338325e+01, 109 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   93.38325    0   35   38.98120   93.38325   140% 

Optimize a model with 165 rows, 102 columns and 512 nonzeros
Model fingerprint: 0x6908c5bd
Variable types: 0 continuous, 102 integer (102 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [6e-01, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 3.6662500
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 163 rows, 102 columns, 506 nonzeros
Variable types: 0 continuous, 102 integer (102 binary)
Found heuristic solution: objective 54.4718000

Root relaxation: objective 1.363255e+02, 116 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  136.32550    0   35   54.47180  136.32550   150%     -    0s
H    0     0                      99.7196500  136.32550  36.7%     -    0s
     0     0  116.11178    0   37   99.71965  116

S_k 1.9272
gamma 0.33
S_k 1.9272
gamma 0.66
S_k 3.8544
gamma 1
S_k 5.84
gamma 1
S_k 5.84
gamma 1
S_k 5.84
gamma 0.33
S_k 1.9272
gamma 0.66
S_k 3.8544
gamma 0.33
S_k 1.9272
gamma 1
S_k 5.84
gamma 0.33
S_k 1.9272
gamma 1
S_k 5.84
gamma 0.33
S_k 1.9272
gamma 0.33
S_k 1.9272
gamma 0.66
S_k 3.8544
gamma -0.33
S_k -1.9272
gamma 0.66
S_k 3.8544
gamma 0.33
S_k 1.9272
gamma 0.33
S_k 1.9272
gamma 1
S_k 5.84
gamma 0.33
S_k 1.9272
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 165 rows, 102 columns and 512 nonzeros
Model fingerprint: 0xe838fc97
Variable types: 0 continuous, 102 integer (102 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [6e-01, 7e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 3.6662500
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 163 rows, 102 colum

[{'z_0': 0.0,
  'z_1': 0.0,
  'z_2': 1.0,
  'z_3': 1.0,
  'z_4': 0.0,
  'z_5': 0.0,
  'z_6': 1.0,
  'z_7': 1.0,
  'z_8': 1.0,
  'z_9': 0.0,
  'z_10': 0.0,
  'z_11': 1.0,
  'z_12': 1.0,
  'z_13': 0.0,
  'z_14': 1.0,
  'z_15': 0.0,
  'z_16': 1.0,
  'z_17': 0.0,
  'z_18': 1.0,
  'z_19': 0.0},
 {'z_0': -0.0,
  'z_1': -0.0,
  'z_2': 1.0,
  'z_3': 1.0,
  'z_4': -0.0,
  'z_5': -0.0,
  'z_6': 1.0,
  'z_7': 1.0,
  'z_8': 1.0,
  'z_9': -0.0,
  'z_10': -0.0,
  'z_11': 1.0,
  'z_12': 1.0,
  'z_13': -0.0,
  'z_14': 1.0,
  'z_15': -0.0,
  'z_16': 1.0,
  'z_17': -0.0,
  'z_18': 1.0,
  'z_19': -0.0,
  'g_0': -0.0,
  'g_1': -0.0,
  'g_2': -0.0,
  'g_3': -0.0,
  'g_4': -0.0,
  'g_5': -0.0,
  'g_6': -0.0,
  'g_7': -0.0,
  'g_8': -0.0,
  'g_9': -0.0,
  'g_10': -0.0,
  'g_11': -0.0,
  'g_12': 1.0,
  'g_13': -0.0,
  'g_14': 1.0,
  'g_15': 1.0,
  'g_16': -0.0,
  'g_17': 1.0,
  'g_18': 1.0,
  'g_19': -0.0,
  'g_20': -0.0,
  'g_21': -0.0,
  'g_22': -0.0,
  'g_23': -0.0,
  'g_24': 1.0,
  'g_25': 1.0,
  'g_26': 

In [135]:
s2 = SolveProblem(budget_const_pct=0.5, inter_subset=inter_subset, inter_effect=inter_effect, project_list=p2)
s2.run()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1 rows, 20 columns and 20 nonzeros
Model fingerprint: 0xd73f89d9
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-01, 9e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 1e+01]
Found heuristic solution: objective 7.7764000
Presolve removed 1 rows and 20 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.04 seconds (0.00 work units)
Thread count was 1 (of 4 available processors)

Solution count 2: 8.33585 7.7764 

Optimal solution found (tolerance 1.00e-04)
Best objective 8.335850000000e+00, best bound 8.335850000000e+00, gap 0.0000%
gamma 0.165
S_k 0.0
gamma 0.165
S_k 0.0
gamma 0.165
S_k 0.0
gamma 0.66
S_k 0.0
gamma 0.33
S_k 0.0
gamma 0.66
S_k 0.0
gamma 0
S_k 0.0
gamma 

gamma 0.66
S_k 0.009636
gamma 0.33
S_k 0.004818
gamma 0.33
S_k 0.004818
gamma 0.33
S_k 0.004818
gamma 0.33
S_k 0.004818
gamma 1
S_k 0.0146
gamma 1
S_k 0.0146
gamma 0.66
S_k 0.009636
gamma 0.33
S_k 0.004818
gamma 0.33
S_k 0.004818
gamma 0.33
S_k 0.004818
gamma 0.33
S_k 0.004818
gamma 0.66
S_k 0.009636
gamma 1
S_k 0.0146
gamma 1
S_k 0.0146
gamma 1
S_k 0.0146
gamma 0.33
S_k 0.004818
gamma 0.66
S_k 0.009636
gamma 0.33
S_k 0.004818
gamma 1
S_k 0.0146
gamma 0.33
S_k 0.004818
gamma 1
S_k 0.0146
gamma 0.33
S_k 0.004818
gamma 0.33
S_k 0.004818
gamma 0.66
S_k 0.009636
gamma -0.33
S_k -0.004818
gamma 0.66
S_k 0.009636
gamma 0.33
S_k 0.004818
gamma 0.33
S_k 0.004818
gamma 1
S_k 0.0146
gamma 0.33
S_k 0.004818
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 165 rows, 102 columns and 512 nonzeros
Model fingerprint: 0x44937ce6
Variable types: 0 continuous, 102 integer (102 binary)
Coefficient 

Presolve time: 0.00s
Presolved: 163 rows, 102 columns, 506 nonzeros
Variable types: 0 continuous, 102 integer (102 binary)
Found heuristic solution: objective 7.9439290

Root relaxation: objective 9.004954e+00, 11 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    9.00495    0   43    7.94393    9.00495  13.4%     -    0s
H    0     0                       8.6833890    9.00495  3.70%     -    0s
     0     0    8.71461    0    1    8.68339    8.71461  0.36%     -    0s
H    0     0                       8.7097870    8.71461  0.06%     -    0s

Cutting planes:
  Gomory: 1
  Cover: 32
  RLT: 16

Explored 1 nodes (75 simplex iterations) in 0.05 seconds (0.00 work units)
Thread count was 4 (of 4 available processors)

Solution count 4: 8.70979 8.68339 7.94393 3.46778 

Optimal solution found (tolerance 1.00e-04)
Best objective 8.709

S_k 0.016863000000000003
gamma 0.66
S_k 0.033726000000000006
gamma 0.33
S_k 0.016863000000000003
gamma 1
S_k 0.051100000000000007
gamma 0.33
S_k 0.016863000000000003
gamma 1
S_k 0.051100000000000007
gamma 0.33
S_k 0.016863000000000003
gamma 0.33
S_k 0.016863000000000003
gamma 0.66
S_k 0.033726000000000006
gamma -0.33
S_k -0.016863000000000003
gamma 0.66
S_k 0.033726000000000006
gamma 0.33
S_k 0.016863000000000003
gamma 0.33
S_k 0.016863000000000003
gamma 1
S_k 0.051100000000000007
gamma 0.33
S_k 0.016863000000000003
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 165 rows, 102 columns and 512 nonzeros
Model fingerprint: 0x00491d4c
Variable types: 0 continuous, 102 integer (102 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [8e-03, 9e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 


Root relaxation: objective 9.907972e+00, 22 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    9.90797    0   37    8.25856    9.90797  20.0%     -    0s
H    0     0                       9.3206145    9.90797  6.30%     -    0s
     0     0    9.44745    0   11    9.32061    9.44745  1.36%     -    0s
     0     0 infeasible    0         9.32061    9.32061  0.00%     -    0s

Cutting planes:
  Gomory: 3
  Cover: 38
  Clique: 3
  RLT: 16

Explored 1 nodes (103 simplex iterations) in 0.15 seconds (0.00 work units)
Thread count was 4 (of 4 available processors)

Solution count 3: 9.32061 8.25856 3.46778 

Optimal solution found (tolerance 1.00e-04)
Best objective 9.320614500000e+00, best bound 9.320614500000e+00, gap 0.0000%
Beta:  0.09
gamma 0.165
S_k 0.012045
gamma 0.165
S_k 0.012045
gamma 0.165
S_k 0.012045
gamma 0.66
S_k 0.04

...

gamma 1
S_k 2.19
gamma 0.33
S_k 0.7227
gamma 0.66
S_k 1.4454
gamma 0.33
S_k 0.7227
gamma 1
S_k 2.19
gamma 0.33
S_k 0.7227
gamma 1
S_k 2.19
gamma 0.33
S_k 0.7227
gamma 0.33
S_k 0.7227
gamma 0.66
S_k 1.4454
gamma -0.33
S_k -0.7227
gamma 0.66
S_k 1.4454
gamma 0.33
S_k 0.7227
gamma 0.33
S_k 0.7227
gamma 1
S_k 2.19
gamma 0.33
S_k 0.7227
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 165 rows, 102 columns and 512 nonzeros
Model fingerprint: 0x22d5afe1
Variable types: 0 continuous, 102 integer (102 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [4e-01, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 3.4677750
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 163 rows, 102 columns, 506 nonzeros
Variable types: 0 continuous, 102 integer (102 binary)
Found heuristic s

Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 165 rows, 102 columns and 512 nonzeros
Model fingerprint: 0x11812d95
Variable types: 0 continuous, 102 integer (102 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [5e-01, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 3.4677750
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 163 rows, 102 columns, 506 nonzeros
Variable types: 0 continuous, 102 integer (102 binary)
Found heuristic solution: objective 49.0701250

Root relaxation: objective 1.145709e+02, 115 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  114.57090    0   35   49.07013  114.57090   133%     -    0s
H    0     0                      84.0660250  11

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 165 rows, 102 columns and 512 nonzeros
Model fingerprint: 0x4647e931
Variable types: 0 continuous, 102 integer (102 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [5e-01, 6e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 3.4677750
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 163 rows, 102 columns, 506 nonzeros
Variable types: 0 continuous, 102 integer (102 binary)
Found heuristic solution: objective 65.5097250

Root relaxation: objective 1.575131e+02, 115 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  157.51315    0   35   65.50973  157.51315   140% 

Variable types: 0 continuous, 102 integer (102 binary)
Found heuristic solution: objective 81.9493250

Root relaxation: objective 2.004554e+02, 123 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  200.45540    0   35   81.94933  200.45540   145%     -    0s
H    0     0                     145.5466250  200.45540  37.7%     -    0s
     0     0  170.65889    0   37  145.54663  170.65889  17.3%     -    0s
     0     0  166.69200    0   37  145.54663  166.69200  14.5%     -    0s
     0     0  159.01663    0   49  145.54663  159.01663  9.25%     -    0s
     0     0  156.20555    0   69  145.54663  156.20555  7.32%     -    0s
     0     0  156.00204    0   79  145.54663  156.00204  7.18%     -    0s
     0     0  155.75874    0   69  145.54663  155.75874  7.02%     -    0s
     0     0  155.75874    0   68  145.54663  155.75874  

[{'z_0': 0.0,
  'z_1': 0.0,
  'z_2': 1.0,
  'z_3': 1.0,
  'z_4': 0.0,
  'z_5': 1.0,
  'z_6': 1.0,
  'z_7': 1.0,
  'z_8': 1.0,
  'z_9': 1.0,
  'z_10': 0.0,
  'z_11': 1.0,
  'z_12': 0.0,
  'z_13': 0.0,
  'z_14': 1.0,
  'z_15': 0.0,
  'z_16': 0.0,
  'z_17': 0.0,
  'z_18': 1.0,
  'z_19': 0.0},
 {'z_0': -0.0,
  'z_1': -0.0,
  'z_2': 1.0,
  'z_3': 1.0,
  'z_4': -0.0,
  'z_5': 1.0,
  'z_6': 1.0,
  'z_7': 1.0,
  'z_8': 1.0,
  'z_9': 1.0,
  'z_10': -0.0,
  'z_11': 1.0,
  'z_12': -0.0,
  'z_13': -0.0,
  'z_14': 1.0,
  'z_15': -0.0,
  'z_16': -0.0,
  'z_17': -0.0,
  'z_18': 1.0,
  'z_19': -0.0,
  'g_0': -0.0,
  'g_1': -0.0,
  'g_2': -0.0,
  'g_3': -0.0,
  'g_4': -0.0,
  'g_5': -0.0,
  'g_6': -0.0,
  'g_7': -0.0,
  'g_8': -0.0,
  'g_9': -0.0,
  'g_10': -0.0,
  'g_11': -0.0,
  'g_12': 1.0,
  'g_13': -0.0,
  'g_14': 1.0,
  'g_15': 1.0,
  'g_16': -0.0,
  'g_17': 1.0,
  'g_18': 1.0,
  'g_19': -0.0,
  'g_20': -0.0,
  'g_21': -0.0,
  'g_22': -0.0,
  'g_23': 1.0,
  'g_24': 1.0,
  'g_25': 1.0,
  'g_26': 1

In [136]:
s3 = SolveProblem(budget_const_pct=0.5, inter_subset=inter_subset, inter_effect=inter_effect, project_list=p3)
s3.run()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1 rows, 20 columns and 20 nonzeros
Model fingerprint: 0x236f84dd
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-01, 9e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 1e+01]
Found heuristic solution: objective 7.6675000
Presolve removed 1 rows and 20 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 4 available processors)

Solution count 2: 8.5025 7.6675 

Optimal solution found (tolerance 1.00e-04)
Best objective 8.502500000000e+00, best bound 8.502500000000e+00, gap 0.0000%
gamma 0.165
S_k 0.0
gamma 0.165
S_k 0.0
gamma 0.165
S_k 0.0
gamma 0.66
S_k 0.0
gamma 0.33
S_k 0.0
gamma 0.66
S_k 0.0
gamma 0
S_k 0.0
gamma 0

S_k 0.004818
gamma 0.66
S_k 0.009636
gamma 1
S_k 0.0146
gamma 1
S_k 0.0146
gamma 1
S_k 0.0146
gamma 0.33
S_k 0.004818
gamma 0.66
S_k 0.009636
gamma 0.33
S_k 0.004818
gamma 1
S_k 0.0146
gamma 0.33
S_k 0.004818
gamma 1
S_k 0.0146
gamma 0.33
S_k 0.004818
gamma 0.33
S_k 0.004818
gamma 0.66
S_k 0.009636
gamma -0.33
S_k -0.004818
gamma 0.66
S_k 0.009636
gamma 0.33
S_k 0.004818
gamma 0.33
S_k 0.004818
gamma 1
S_k 0.0146
gamma 0.33
S_k 0.004818
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 165 rows, 102 columns and 512 nonzeros
Model fingerprint: 0xf382dc65
Variable types: 0 continuous, 102 integer (102 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [2e-03, 9e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 3.0825000
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 1

H    0     0                       8.8502720    9.12833  3.14%     -    0s
     0     0 infeasible    0         8.85027    8.85027  0.00%     -    0s

Cutting planes:
  Gomory: 1
  Cover: 24
  RLT: 13

Explored 1 nodes (74 simplex iterations) in 0.08 seconds (0.00 work units)
Thread count was 4 (of 4 available processors)

Solution count 3: 8.85027 8.10056 3.0825 

Optimal solution found (tolerance 1.00e-04)
Best objective 8.850272000000e+00, best bound 8.850272000000e+00, gap 0.0000%
Beta:  0.04
gamma 0.165
S_k 0.0060225
gamma 0.165
S_k 0.0060225
gamma 0.165
S_k 0.0060225
gamma 0.66
S_k 0.02409
gamma 0.33
S_k 0.012045
gamma 0.66
S_k 0.02409
gamma 0
S_k 0.0
gamma 0.66
S_k 0.02409
gamma 0.66
S_k 0.02409
gamma 1.165
S_k 0.042522500000000005
gamma 1
S_k 0.0365
gamma 0.66
S_k 0.02409
gamma 0.33
S_k 0.012045
gamma 0.33
S_k 0.012045
gamma 0.33
S_k 0.012045
gamma 0.33
S_k 0.012045
gamma 0.33
S_k 0.012045
gamma 0.33
S_k 0.012045
gamma 0.33
S_k 0.012045
gamma 0.33
S_k 0.012045
gamma 1
S_k 0.036

gamma 0.33
S_k 0.016863000000000003
gamma 0.33
S_k 0.016863000000000003
gamma 0.33
S_k 0.016863000000000003
gamma 1
S_k 0.051100000000000007
gamma 1
S_k 0.051100000000000007
gamma 0.66
S_k 0.033726000000000006
gamma 0.33
S_k 0.016863000000000003
gamma 0.33
S_k 0.016863000000000003
gamma 0.33
S_k 0.016863000000000003
gamma 0.33
S_k 0.016863000000000003
gamma 0.66
S_k 0.033726000000000006
gamma 1
S_k 0.051100000000000007
gamma 1
S_k 0.051100000000000007
gamma 1
S_k 0.051100000000000007
gamma 0.33
S_k 0.016863000000000003
gamma 0.66
S_k 0.033726000000000006
gamma 0.33
S_k 0.016863000000000003
gamma 1
S_k 0.051100000000000007
gamma 0.33
S_k 0.016863000000000003
gamma 1
S_k 0.051100000000000007
gamma 0.33
S_k 0.016863000000000003
gamma 0.33
S_k 0.016863000000000003
gamma 0.66
S_k 0.033726000000000006
gamma -0.33
S_k -0.016863000000000003
gamma 0.66
S_k 0.033726000000000006
gamma 0.33
S_k 0.016863000000000003
gamma 0.33
S_k 0.016863000000000003
gamma 1
S_k 0.051100000000000007
gamma 0.33
S_k

  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 3.0825000
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 163 rows, 102 columns, 506 nonzeros
Variable types: 0 continuous, 102 integer (102 binary)
Found heuristic solution: objective 8.5356440

Root relaxation: objective 9.922758e+00, 22 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    9.92276    0   40    8.53564    9.92276  16.3%     -    0s
H    0     0                       9.2531110    9.92276  7.24%     -    0s
     0     0    9.38378    0   24    9.25311    9.38378  1.41%     -    0s
*    0     0               0       9.2849870    9.28499  0.00%     -    0s

Cutting planes:
  Gomory: 7
  Cover: 40
  MIR: 2
  Zero half: 3
  RLT: 23

Explored 1 nodes (159 simplex iterations) in 0.06 seconds (0.01

...

Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 165 rows, 102 columns and 512 nonzeros
Model fingerprint: 0x6b1b2c0a
Variable types: 0 continuous, 102 integer (102 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [3e-01, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 3.0825000
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 163 rows, 102 columns, 506 nonzeros
Variable types: 0 continuous, 102 integer (102 binary)
Found heuristic solution: objective 32.4119000

Root relaxation: objective 7.114548e+01, 128 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   71.14548    0   40   32.41190   71.14548   120%     -    0s
H    0     0                      50.3021500   7


Root relaxation: objective 1.140206e+02, 123 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  114.02063    0   35   48.85150  114.02063   133%     -    0s
H    0     0                      83.5157500  114.02063  36.5%     -    0s
     0     0   97.78774    0   60   83.51575   97.78774  17.1%     -    0s
     0     0   95.83566    0   55   83.51575   95.83566  14.8%     -    0s
     0     0   95.42783    0   38   83.51575   95.42783  14.3%     -    0s
     0     0   95.41051    0   67   83.51575   95.41051  14.2%     -    0s
     0     0   95.40759    0   67   83.51575   95.40759  14.2%     -    0s
     0     0   89.81425    0    8   83.51575   89.81425  7.54%     -    0s
     0     0   87.51890    0   32   83.51575   87.51890  4.79%     -    0s
     0     0   87.21124    0   86   83.51575   87.21124  4.42%     -    0s
     0   

Optimize a model with 165 rows, 102 columns and 512 nonzeros
Model fingerprint: 0xb3bcdf32
Variable types: 0 continuous, 102 integer (102 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [3e-01, 6e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 3.0825000
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 163 rows, 102 columns, 506 nonzeros
Variable types: 0 continuous, 102 integer (102 binary)
Found heuristic solution: objective 65.2911000

Root relaxation: objective 1.569629e+02, 119 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  156.96287    0   35   65.29110  156.96287   140%     -    0s
H    0     0                     114.2560500  156.96287  37.4%     -    0s
     0     0  133.53809    0   57  114.25605  133

 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  199.90512    0   35   81.73070  199.90512   145%     -    0s
H    0     0                     144.9963500  199.90512  37.9%     -    0s
     0     0  170.30060    0   50  144.99635  170.30060  17.5%     -    0s
     0     0  166.49229    0   44  144.99635  166.49229  14.8%     -    0s
     0     0  152.21610    0    3  144.99635  152.21610  4.98%     -    0s
H    0     0                     148.9639500  152.21610  2.18%     -    0s
     0     0  151.13205    0    2  148.96395  151.13205  1.46%     -    0s
     0     0 infeasible    0       148.96395  148.96395  0.00%     -    0s

Cutting planes:
  Gomory: 14
  Cover: 40
  Clique: 10
  Zero half: 5
  RLT: 35

Explored 1 nodes (331 simplex iterations) in 0.07 seconds (0.01 work units)
Thread count was 4 (of 4 available processors)

Solution count 4: 148.964 144.996 81.7307 3.0825 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.48963

[{'z_0': 0.0,
  'z_1': 0.0,
  'z_2': 1.0,
  'z_3': 1.0,
  'z_4': 0.0,
  'z_5': 1.0,
  'z_6': 1.0,
  'z_7': 1.0,
  'z_8': 1.0,
  'z_9': 1.0,
  'z_10': 0.0,
  'z_11': 0.0,
  'z_12': 0.0,
  'z_13': 0.0,
  'z_14': 1.0,
  'z_15': 0.0,
  'z_16': 0.0,
  'z_17': 0.0,
  'z_18': 1.0,
  'z_19': 1.0},
 {'z_0': -0.0,
  'z_1': -0.0,
  'z_2': 1.0,
  'z_3': 1.0,
  'z_4': -0.0,
  'z_5': 1.0,
  'z_6': 1.0,
  'z_7': 1.0,
  'z_8': 1.0,
  'z_9': 1.0,
  'z_10': -0.0,
  'z_11': -0.0,
  'z_12': -0.0,
  'z_13': -0.0,
  'z_14': 1.0,
  'z_15': -0.0,
  'z_16': -0.0,
  'z_17': -0.0,
  'z_18': 1.0,
  'z_19': 1.0,
  'g_0': -0.0,
  'g_1': -0.0,
  'g_2': -0.0,
  'g_3': -0.0,
  'g_4': -0.0,
  'g_5': -0.0,
  'g_6': -0.0,
  'g_7': -0.0,
  'g_8': -0.0,
  'g_9': -0.0,
  'g_10': -0.0,
  'g_11': -0.0,
  'g_12': 1.0,
  'g_13': -0.0,
  'g_14': 1.0,
  'g_15': 1.0,
  'g_16': 1.0,
  'g_17': 1.0,
  'g_18': 1.0,
  'g_19': -0.0,
  'g_20': -0.0,
  'g_21': -0.0,
  'g_22': -0.0,
  'g_23': 1.0,
  'g_24': 1.0,
  'g_25': 1.0,
  'g_26': 1.

In [137]:
s4 = SolveProblem(budget_const_pct=0.25, inter_subset=inter_subset, inter_effect=inter_effect, project_list=p1)
s4.run()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1 rows, 20 columns and 20 nonzeros
Model fingerprint: 0x29450bcd
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+00, 5e+00]
Found heuristic solution: objective 3.6250000
Presolve removed 1 rows and 20 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 4 available processors)

Solution count 2: 4.45875 3.625 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.458750000000e+00, best bound 4.458750000000e+00, gap 0.0000%
gamma 0.165
S_k 0.0
gamma 0.165
S_k 0.0
gamma 0.165
S_k 0.0
gamma 0.66
S_k 0.0
gamma 0.33
S_k 0.0
gamma 0.66
S_k 0.0
gamma 0
S_k 0.0
gamma 0

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 165 rows, 102 columns and 512 nonzeros
Model fingerprint: 0xba44151d
Variable types: 0 continuous, 102 integer (102 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [2e-03, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 3.6662500
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 163 rows, 102 columns, 506 nonzeros
Variable types: 0 continuous, 102 integer (102 binary)
Found heuristic solution: objective 3.8662260

Root relaxation: objective 4.663114e+00, 3 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    4.66311    0   35    3.86623    4.66311  20.6%    

Best objective 4.603874000000e+00, best bound 4.603874000000e+00, gap 0.0000%
Beta:  0.04
gamma 0.165
S_k 0.0060225
gamma 0.165
S_k 0.0060225
gamma 0.165
S_k 0.0060225
gamma 0.66
S_k 0.02409
gamma 0.33
S_k 0.012045
gamma 0.66
S_k 0.02409
gamma 0
S_k 0.0
gamma 0.66
S_k 0.02409
gamma 0.66
S_k 0.02409
gamma 1.165
S_k 0.042522500000000005
gamma 1
S_k 0.0365
gamma 0.66
S_k 0.02409
gamma 0.33
S_k 0.012045
gamma 0.33
S_k 0.012045
gamma 0.33
S_k 0.012045
gamma 0.33
S_k 0.012045
gamma 0.33
S_k 0.012045
gamma 0.33
S_k 0.012045
gamma 0.33
S_k 0.012045
gamma 0.33
S_k 0.012045
gamma 1
S_k 0.0365
gamma 1
S_k 0.0365
gamma 0.33
S_k 0.012045
gamma 0.66
S_k 0.02409
gamma 0.33
S_k 0.012045
gamma 1
S_k 0.0365
gamma 0.33
S_k 0.012045
gamma 0.33
S_k 0.012045
gamma 0.66
S_k 0.02409
gamma 0.66
S_k 0.02409
gamma 0.33
S_k 0.012045
gamma 1
S_k 0.0365
gamma 1
S_k 0.0365
gamma 0.33
S_k 0.012045
gamma 0.33
S_k 0.012045
gamma 0.33
S_k 0.012045
gamma 0.66
S_k 0.02409
gamma -0.33
S_k -0.012045
gamma 0.33
S_k 0.012045


Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 165 rows, 102 columns and 512 nonzeros
Model fingerprint: 0x5efa9999
Variable types: 0 continuous, 102 integer (102 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [8e-03, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 3.6662500
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 163 rows, 102 columns, 506 nonzeros
Variable types: 0 continuous, 102 integer (102 binary)
Found heuristic solution: objective 3.9505410

Root relaxation: objective 5.205120e+00, 22 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    5.20512    0   39    3.95054    5.20512  31.8%   


Root relaxation: objective 5.442296e+00, 31 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    5.44230    0   39    3.98427    5.44230  36.6%     -    0s
H    0     0                       4.7015290    5.44230  15.8%     -    0s
     0     0    4.94091    0   33    4.70153    4.94091  5.09%     -    0s
H    0     0                       4.8084790    4.94091  2.75%     -    0s
     0     0    4.92406    0   35    4.80848    4.92406  2.40%     -    0s
     0     0     cutoff    0         4.80848    4.80848  0.00%     -    0s

Cutting planes:
  Gomory: 5
  Cover: 49
  Zero half: 1
  RLT: 28
  BQP: 1

Explored 1 nodes (131 simplex iterations) in 0.08 seconds (0.00 work units)
Thread count was 4 (of 4 available processors)

Solution count 4: 4.80848 4.70153 3.98427 3.66625 

Optimal solution found (tolerance 1.00e-04)
Best objective

...

Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 165 rows, 102 columns and 512 nonzeros
Model fingerprint: 0x5e8de2f0
Variable types: 0 continuous, 102 integer (102 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [4e-01, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 3.6662500
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 163 rows, 102 columns, 506 nonzeros
Variable types: 0 continuous, 102 integer (102 binary)
Found heuristic solution: objective 10.3368000

Root relaxation: objective 4.356458e+01, 120 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   43.56458    0   42   10.33680   43.56458   321%     -    0s
H    0     0                      20.1808500   4

Optimize a model with 165 rows, 102 columns and 512 nonzeros
Model fingerprint: 0xacdadacf
Variable types: 0 continuous, 102 integer (102 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [6e-01, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 3.6662500
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 163 rows, 102 columns, 506 nonzeros
Variable types: 0 continuous, 102 integer (102 binary)
Found heuristic solution: objective 14.6730000

Root relaxation: objective 7.005262e+01, 114 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   70.05262    0   42   14.67300   70.05262   377%     -    0s
H    0     0                      31.0797500   70.05262   125%     -    0s
     0     0   49.66287    0   54   31.07975   49

Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 165 rows, 102 columns and 512 nonzeros
Model fingerprint: 0xb6f8daf1
Variable types: 0 continuous, 102 integer (102 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [6e-01, 6e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 3.6662500
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 163 rows, 102 columns, 506 nonzeros
Variable types: 0 continuous, 102 integer (102 binary)
Found heuristic solution: objective 19.0092000

Root relaxation: objective 9.654067e+01, 113 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   96.54067    0   42   19.00920   96.54067   408%     -    0s
H    0     0                      41.9786500   9

Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 165 rows, 102 columns and 512 nonzeros
Model fingerprint: 0xfaae3edd
Variable types: 0 continuous, 102 integer (102 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [6e-01, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 3.6662500
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 163 rows, 102 columns, 506 nonzeros
Variable types: 0 continuous, 102 integer (102 binary)
Found heuristic solution: objective 23.3454000

Root relaxation: objective 1.230287e+02, 113 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  123.02873    0   42   23.34540  123.02873   427%     -    0s
H    0     0                      52.8775500  12

[{'z_0': 0.0,
  'z_1': 0.0,
  'z_2': 1.0,
  'z_3': 0.0,
  'z_4': 0.0,
  'z_5': 0.0,
  'z_6': 1.0,
  'z_7': 1.0,
  'z_8': 1.0,
  'z_9': 0.0,
  'z_10': 0.0,
  'z_11': 0.0,
  'z_12': 0.0,
  'z_13': 0.0,
  'z_14': 1.0,
  'z_15': 0.0,
  'z_16': 0.0,
  'z_17': 0.0,
  'z_18': 0.0,
  'z_19': 0.0},
 {'z_0': -0.0,
  'z_1': -0.0,
  'z_2': 1.0,
  'z_3': -0.0,
  'z_4': -0.0,
  'z_5': -0.0,
  'z_6': 1.0,
  'z_7': 1.0,
  'z_8': 1.0,
  'z_9': -0.0,
  'z_10': -0.0,
  'z_11': -0.0,
  'z_12': -0.0,
  'z_13': -0.0,
  'z_14': 1.0,
  'z_15': -0.0,
  'z_16': -0.0,
  'z_17': -0.0,
  'z_18': -0.0,
  'z_19': -0.0,
  'g_0': -0.0,
  'g_1': -0.0,
  'g_2': -0.0,
  'g_3': -0.0,
  'g_4': -0.0,
  'g_5': -0.0,
  'g_6': -0.0,
  'g_7': -0.0,
  'g_8': -0.0,
  'g_9': -0.0,
  'g_10': -0.0,
  'g_11': -0.0,
  'g_12': -0.0,
  'g_13': -0.0,
  'g_14': 1.0,
  'g_15': 1.0,
  'g_16': -0.0,
  'g_17': -0.0,
  'g_18': -0.0,
  'g_19': -0.0,
  'g_20': -0.0,
  'g_21': -0.0,
  'g_22': -0.0,
  'g_23': -0.0,
  'g_24': 1.0,
  'g_25': 1.0,
  

In [138]:
s5 = SolveProblem(budget_const_pct=0.25,inter_subset=inter_subset, inter_effect=inter_effect, project_list=p2)
s5.run()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1 rows, 20 columns and 20 nonzeros
Model fingerprint: 0x67676d0a
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-01, 9e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+00, 5e+00]
Found heuristic solution: objective 3.5544500
Presolve removed 1 rows and 20 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.04 seconds (0.00 work units)
Thread count was 1 (of 4 available processors)

Solution count 2: 4.47363 3.55445 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.473625000000e+00, best bound 4.473625000000e+00, gap 0.0000%
gamma 0.165
S_k 0.0
gamma 0.165
S_k 0.0
gamma 0.165
S_k 0.0
gamma 0.66
S_k 0.0
gamma 0.33
S_k 0.0
gamma 0.66
S_k 0.0
gamma 0
S_k 0.0
gamma

S_k 0.009636
gamma 0.33
S_k 0.004818
gamma 1
S_k 0.0146
gamma 0.33
S_k 0.004818
gamma 1
S_k 0.0146
gamma 0.33
S_k 0.004818
gamma 0.33
S_k 0.004818
gamma 0.66
S_k 0.009636
gamma -0.33
S_k -0.004818
gamma 0.66
S_k 0.009636
gamma 0.33
S_k 0.004818
gamma 0.33
S_k 0.004818
gamma 1
S_k 0.0146
gamma 0.33
S_k 0.004818
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 165 rows, 102 columns and 512 nonzeros
Model fingerprint: 0x57edd680
Variable types: 0 continuous, 102 integer (102 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [2e-03, 9e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 3.4677750
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 163 rows, 102 columns, 506 nonzeros
Variable types: 0 continuous, 102 integer (102 binary)
Found heuristic solution: objective 3.8

     0     0    4.61875    0    2    4.56180    4.61875  1.25%     -    0s
H    0     0                       4.6091130    4.61875  0.21%     -    0s
     0     0    4.61875    0    2    4.60911    4.61875  0.21%     -    0s

Cutting planes:
  Gomory: 4
  Cover: 27
  RLT: 11

Explored 1 nodes (48 simplex iterations) in 0.10 seconds (0.00 work units)
Thread count was 4 (of 4 available processors)

Solution count 4: 4.60911 4.5618 3.8719 3.46778 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.609113000000e+00, best bound 4.609113000000e+00, gap 0.0000%
Beta:  0.04
gamma 0.165
S_k 0.0060225
gamma 0.165
S_k 0.0060225
gamma 0.165
S_k 0.0060225
gamma 0.66
S_k 0.02409
gamma 0.33
S_k 0.012045
gamma 0.66
S_k 0.02409
gamma 0
S_k 0.0
gamma 0.66
S_k 0.02409
gamma 0.66
S_k 0.02409
gamma 1.165
S_k 0.042522500000000005
gamma 1
S_k 0.0365
gamma 0.66
S_k 0.02409
gamma 0.33
S_k 0.012045
gamma 0.33
S_k 0.012045
gamma 0.33
S_k 0.012045
gamma 0.33
S_k 0.012045
gamma 0.33
S_k 0.012045
gamma 0

gamma 0.33
S_k 0.016863000000000003
gamma 0.33
S_k 0.016863000000000003
gamma 0.33
S_k 0.016863000000000003
gamma 0.33
S_k 0.016863000000000003
gamma 0.33
S_k 0.016863000000000003
gamma 1
S_k 0.051100000000000007
gamma 0.33
S_k 0.016863000000000003
gamma 0.66
S_k 0.033726000000000006
gamma 0.33
S_k 0.016863000000000003
gamma 0.33
S_k 0.016863000000000003
gamma 0.33
S_k 0.016863000000000003
gamma 0.33
S_k 0.016863000000000003
gamma 1
S_k 0.051100000000000007
gamma 1
S_k 0.051100000000000007
gamma 0.66
S_k 0.033726000000000006
gamma 0.33
S_k 0.016863000000000003
gamma 0.33
S_k 0.016863000000000003
gamma 0.33
S_k 0.016863000000000003
gamma 0.33
S_k 0.016863000000000003
gamma 0.66
S_k 0.033726000000000006
gamma 1
S_k 0.051100000000000007
gamma 1
S_k 0.051100000000000007
gamma 1
S_k 0.051100000000000007
gamma 0.33
S_k 0.016863000000000003
gamma 0.66
S_k 0.033726000000000006
gamma 0.33
S_k 0.016863000000000003
gamma 1
S_k 0.051100000000000007
gamma 0.33
S_k 0.016863000000000003
gamma 1
S_k 0

gamma 1
S_k 0.0657
gamma 0.33
S_k 0.021681
gamma 0.33
S_k 0.021681
gamma 0.66
S_k 0.043362
gamma -0.33
S_k -0.021681
gamma 0.66
S_k 0.043362
gamma 0.33
S_k 0.021681
gamma 0.33
S_k 0.021681
gamma 1
S_k 0.0657
gamma 0.33
S_k 0.021681
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 165 rows, 102 columns and 512 nonzeros
Model fingerprint: 0x2488f48c
Variable types: 0 continuous, 102 integer (102 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e-02, 9e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 3.4677750
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 163 rows, 102 columns, 506 nonzeros
Variable types: 0 continuous, 102 integer (102 binary)
Found heuristic solution: objective 3.9562170

Root relaxation: objective 5.339349e+00, 30 iterations, 0.00 seconds (0.

...

Variable types: 0 continuous, 102 integer (102 binary)
Found heuristic solution: objective 6.9823250

Root relaxation: objective 3.023555e+01, 114 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   30.23555    0   42    6.98232   30.23555   333%     -    0s
H    0     0                      14.6464000   30.23555   106%     -    0s
     0     0   21.88203    0   52   14.64640   21.88203  49.4%     -    0s
     0     0   21.42578    0   51   14.64640   21.42578  46.3%     -    0s
     0     0   17.46624    0   58   14.64640   17.46624  19.3%     -    0s
     0     0   17.02099    0   53   14.64640   17.02099  16.2%     -    0s
     0     0   16.90807    0   36   14.64640   16.90807  15.4%     -    0s
     0     0   16.89751    0   55   14.64640   16.89751  15.4%     -    0s
     0     0   16.89400    0   54   14.64640   16.89400  1

  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 3.4677750
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 163 rows, 102 columns, 506 nonzeros
Variable types: 0 continuous, 102 integer (102 binary)
Found heuristic solution: objective 10.3549250

Root relaxation: objective 5.672360e+01, 118 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   56.72360    0   42   10.35492   56.72360   448%     -    0s
H    0     0                      25.5453000   56.72360   122%     -    0s
     0     0   40.00220    0   57   25.54530   40.00220  56.6%     -    0s
     0     0   39.19109    0   56   25.54530   39.19109  53.4%     -    0s
     0     0   32.19242    0   56   25.54530   32.19242  26.0%     -    0s
     0     0   30.88374    0   55   25.54530   30.88374  20.9%     -    0s
     0     0   30.568

Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 165 rows, 102 columns and 512 nonzeros
Model fingerprint: 0x99dab299
Variable types: 0 continuous, 102 integer (102 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [5e-01, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 3.4677750
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 163 rows, 102 columns, 506 nonzeros
Variable types: 0 continuous, 102 integer (102 binary)
Found heuristic solution: objective 13.7275250

Root relaxation: objective 8.321165e+01, 122 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   83.21165    0   42   13.72752   83.21165   506%     -    0s
H    0     0                      36.4442000   8

gamma 0.33
S_k 1.9272
gamma 0.66
S_k 3.8544
gamma 0.33
S_k 1.9272
gamma 0.33
S_k 1.9272
gamma 0.33
S_k 1.9272
gamma 0.33
S_k 1.9272
gamma 0.33
S_k 1.9272
gamma 1
S_k 5.84
gamma 0.33
S_k 1.9272
gamma 0.66
S_k 3.8544
gamma 0.33
S_k 1.9272
gamma 0.33
S_k 1.9272
gamma 0.33
S_k 1.9272
gamma 0.33
S_k 1.9272
gamma 1
S_k 5.84
gamma 1
S_k 5.84
gamma 0.66
S_k 3.8544
gamma 0.33
S_k 1.9272
gamma 0.33
S_k 1.9272
gamma 0.33
S_k 1.9272
gamma 0.33
S_k 1.9272
gamma 0.66
S_k 3.8544
gamma 1
S_k 5.84
gamma 1
S_k 5.84
gamma 1
S_k 5.84
gamma 0.33
S_k 1.9272
gamma 0.66
S_k 3.8544
gamma 0.33
S_k 1.9272
gamma 1
S_k 5.84
gamma 0.33
S_k 1.9272
gamma 1
S_k 5.84
gamma 0.33
S_k 1.9272
gamma 0.33
S_k 1.9272
gamma 0.66
S_k 3.8544
gamma -0.33
S_k -1.9272
gamma 0.66
S_k 3.8544
gamma 0.33
S_k 1.9272
gamma 0.33
S_k 1.9272
gamma 1
S_k 5.84
gamma 0.33
S_k 1.9272
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 165 r

[{'z_0': 0.0,
  'z_1': 0.0,
  'z_2': 1.0,
  'z_3': 0.0,
  'z_4': 0.0,
  'z_5': 0.0,
  'z_6': 1.0,
  'z_7': 1.0,
  'z_8': 1.0,
  'z_9': 0.0,
  'z_10': 0.0,
  'z_11': 0.0,
  'z_12': 0.0,
  'z_13': 0.0,
  'z_14': 1.0,
  'z_15': 0.0,
  'z_16': 0.0,
  'z_17': 0.0,
  'z_18': 0.0,
  'z_19': 0.0},
 {'z_0': -0.0,
  'z_1': -0.0,
  'z_2': 1.0,
  'z_3': -0.0,
  'z_4': -0.0,
  'z_5': -0.0,
  'z_6': 1.0,
  'z_7': 1.0,
  'z_8': 1.0,
  'z_9': -0.0,
  'z_10': -0.0,
  'z_11': -0.0,
  'z_12': -0.0,
  'z_13': -0.0,
  'z_14': 1.0,
  'z_15': -0.0,
  'z_16': -0.0,
  'z_17': -0.0,
  'z_18': -0.0,
  'z_19': -0.0,
  'g_0': -0.0,
  'g_1': -0.0,
  'g_2': -0.0,
  'g_3': -0.0,
  'g_4': -0.0,
  'g_5': -0.0,
  'g_6': -0.0,
  'g_7': -0.0,
  'g_8': -0.0,
  'g_9': -0.0,
  'g_10': -0.0,
  'g_11': -0.0,
  'g_12': -0.0,
  'g_13': -0.0,
  'g_14': 1.0,
  'g_15': 1.0,
  'g_16': -0.0,
  'g_17': -0.0,
  'g_18': -0.0,
  'g_19': -0.0,
  'g_20': -0.0,
  'g_21': -0.0,
  'g_22': -0.0,
  'g_23': -0.0,
  'g_24': 1.0,
  'g_25': 1.0,
  

In [139]:
s6 = SolveProblem(budget_const_pct=0.75, inter_subset=inter_subset, inter_effect=inter_effect, project_list=p2)
s6.run()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1 rows, 20 columns and 20 nonzeros
Model fingerprint: 0x0e69e56f
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-01, 9e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 2e+01]
Found heuristic solution: objective 11.4389000
Presolve removed 1 rows and 20 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 4 available processors)

Solution count 2: 11.7483 11.4389 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.174835000000e+01, best bound 1.174835000000e+01, gap 0.0000%
gamma 0.165
S_k 0.0
gamma 0.165
S_k 0.0
gamma 0.165
S_k 0.0
gamma 0.66
S_k 0.0
gamma 0.33
S_k 0.0
gamma 0.66
S_k 0.0
gamma 0
S_k 0.0
gamm

gamma 0.33
S_k 0.004818
gamma 1
S_k 0.0146
gamma 0.33
S_k 0.004818
gamma 1
S_k 0.0146
gamma 0.33
S_k 0.004818
gamma 0.33
S_k 0.004818
gamma 0.66
S_k 0.009636
gamma -0.33
S_k -0.004818
gamma 0.66
S_k 0.009636
gamma 0.33
S_k 0.004818
gamma 0.33
S_k 0.004818
gamma 1
S_k 0.0146
gamma 0.33
S_k 0.004818
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 165 rows, 102 columns and 512 nonzeros
Model fingerprint: 0x3bbfee56
Variable types: 0 continuous, 102 integer (102 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [2e-03, 9e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 3.4677750
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 163 rows, 102 columns, 506 nonzeros
Variable types: 0 continuous, 102 integer (102 binary)
Found heuristic solution: objective 11.6170590

Root

H    0     0                      12.4697260   12.67551  1.65%     -    0s
     0     0   12.47483    0   16   12.46973   12.47483  0.04%     -    0s

Cutting planes:
  Gomory: 1
  Cover: 24
  RLT: 4

Explored 1 nodes (65 simplex iterations) in 0.08 seconds (0.00 work units)
Thread count was 4 (of 4 available processors)

Solution count 3: 12.4697 11.9899 3.46778 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.246972600000e+01, best bound 1.246972600000e+01, gap 0.0000%
Beta:  0.04
gamma 0.165
S_k 0.0060225
gamma 0.165
S_k 0.0060225
gamma 0.165
S_k 0.0060225
gamma 0.66
S_k 0.02409
gamma 0.33
S_k 0.012045
gamma 0.66
S_k 0.02409
gamma 0
S_k 0.0
gamma 0.66
S_k 0.02409
gamma 0.66
S_k 0.02409
gamma 1.165
S_k 0.042522500000000005
gamma 1
S_k 0.0365
gamma 0.66
S_k 0.02409
gamma 0.33
S_k 0.012045
gamma 0.33
S_k 0.012045
gamma 0.33
S_k 0.012045
gamma 0.33
S_k 0.012045
gamma 0.33
S_k 0.012045
gamma 0.33
S_k 0.012045
gamma 0.33
S_k 0.012045
gamma 0.33
S_k 0.012045
gamma 1
S_k 0.036

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 165 rows, 102 columns and 512 nonzeros
Model fingerprint: 0x7ff7a5a3
Variable types: 0 continuous, 102 integer (102 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [8e-03, 9e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 3.4677750
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 163 rows, 102 columns, 506 nonzeros
Variable types: 0 continuous, 102 integer (102 binary)
Found heuristic solution: objective 12.5148580

Root relaxation: objective 1.341360e+01, 16 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   13.41360    0   28   12.51486   13.41360  7.18%  

     0     0   13.59357    0   11   13.44263   13.59357  1.12%     -    0s
H    0     0                      13.5043395   13.59357  0.66%     -    0s
     0     0   13.56380    0   39   13.50434   13.56380  0.44%     -    0s
     0     0 infeasible    0        13.50434   13.50434  0.00%     -    0s

Explored 1 nodes (106 simplex iterations) in 0.05 seconds (0.00 work units)
Thread count was 4 (of 4 available processors)

Solution count 5: 13.5043 13.4426 13.2259 ... 3.46778

Optimal solution found (tolerance 1.00e-04)
Best objective 1.350433950000e+01, best bound 1.350433950000e+01, gap 0.0000%
Beta:  0.09
gamma 0.165
S_k 0.012045
gamma 0.165
S_k 0.012045
gamma 0.165
S_k 0.012045
gamma 0.66
S_k 0.04818
gamma 0.33
S_k 0.02409
gamma 0.66
S_k 0.04818
gamma 0
S_k 0.0
gamma 0.66
S_k 0.04818
gamma 0.66
S_k 0.04818
gamma 1.165
S_k 0.08504500000000001
gamma 1
S_k 0.073
gamma 0.66
S_k 0.04818
gamma 0.33
S_k 0.02409
gamma 0.33
S_k 0.02409
gamma 0.33
S_k 0.02409
gamma 0.33
S_k 0.02409
gamma 0.33


...


    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   64.92878    0   18   49.89297   64.92878  30.1%     -    0s
H    0     0                      59.7421250   64.92878  8.68%     -    0s
     0     0   63.32185    0   16   59.74213   63.32185  5.99%     -    0s
     0     0   61.48819    0   20   59.74213   61.48819  2.92%     -    0s
     0     0   60.95397    0    5   59.74213   60.95397  2.03%     -    0s
     0     0   60.19970    0    1   59.74213   60.19970  0.77%     -    0s
H    0     1                      59.9588000   60.19970  0.40%     -    0s

Cutting planes:
  Gomory: 1
  Cover: 23
  Clique: 4
  RLT: 8

Explored 1 nodes (217 simplex iterations) in 0.44 seconds (0.01 work units)
Thread count was 4 (of 4 available processors)

Solution count 4: 59.9588 59.7421 49.893 3.46778 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.995880000000e+01, best b

S_k 2.409
gamma 0.33
S_k 1.2045
gamma 0.33
S_k 1.2045
gamma 0.33
S_k 1.2045
gamma 0.33
S_k 1.2045
gamma 0.66
S_k 2.409
gamma 1
S_k 3.65
gamma 1
S_k 3.65
gamma 1
S_k 3.65
gamma 0.33
S_k 1.2045
gamma 0.66
S_k 2.409
gamma 0.33
S_k 1.2045
gamma 1
S_k 3.65
gamma 0.33
S_k 1.2045
gamma 1
S_k 3.65
gamma 0.33
S_k 1.2045
gamma 0.33
S_k 1.2045
gamma 0.66
S_k 2.409
gamma -0.33
S_k -1.2045
gamma 0.66
S_k 2.409
gamma 0.33
S_k 1.2045
gamma 0.33
S_k 1.2045
gamma 1
S_k 3.65
gamma 0.33
S_k 1.2045
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 165 rows, 102 columns and 512 nonzeros
Model fingerprint: 0x5c21c5c2
Variable types: 0 continuous, 102 integer (102 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [5e-01, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 3.4677750
Presolve removed 2 rows and 

Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 165 rows, 102 columns and 512 nonzeros
Model fingerprint: 0x0ea56e83
Variable types: 0 continuous, 102 integer (102 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [5e-01, 6e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 3.4677750
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 163 rows, 102 columns, 506 nonzeros
Variable types: 0 continuous, 102 integer (102 binary)
Found heuristic solution: objective 136.6238750

Root relaxation: objective 2.000841e+02, 112 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  200.08413    0   18  136.62387  200.08413  46.4%     -    0s
H    0     0                     182.7995500  2


Cutting planes:
  Gomory: 5
  Cover: 34
  Clique: 8
  RLT: 8

Explored 1 nodes (226 simplex iterations) in 0.14 seconds (0.01 work units)
Thread count was 4 (of 4 available processors)

Solution count 3: 231.936 172.423 3.46778 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.319358500000e+02, best bound 2.319358500000e+02, gap 0.0000%
Beta:  9


[{'z_0': 1.0,
  'z_1': 1.0,
  'z_2': 1.0,
  'z_3': 1.0,
  'z_4': 0.0,
  'z_5': 1.0,
  'z_6': 1.0,
  'z_7': 1.0,
  'z_8': 1.0,
  'z_9': 1.0,
  'z_10': 0.0,
  'z_11': 1.0,
  'z_12': 1.0,
  'z_13': 0.0,
  'z_14': 1.0,
  'z_15': 0.0,
  'z_16': 1.0,
  'z_17': 0.0,
  'z_18': 1.0,
  'z_19': 1.0},
 {'z_0': 1.0,
  'z_1': 1.0,
  'z_2': 1.0,
  'z_3': 1.0,
  'z_4': -0.0,
  'z_5': 1.0,
  'z_6': 1.0,
  'z_7': 1.0,
  'z_8': 1.0,
  'z_9': 1.0,
  'z_10': -0.0,
  'z_11': 1.0,
  'z_12': 1.0,
  'z_13': -0.0,
  'z_14': 1.0,
  'z_15': -0.0,
  'z_16': 1.0,
  'z_17': -0.0,
  'z_18': 1.0,
  'z_19': 1.0,
  'g_0': 1.0,
  'g_1': 1.0,
  'g_2': 1.0,
  'g_3': 1.0,
  'g_4': 1.0,
  'g_5': -0.0,
  'g_6': 1.0,
  'g_7': 1.0,
  'g_8': 1.0,
  'g_9': 1.0,
  'g_10': 1.0,
  'g_11': 1.0,
  'g_12': 1.0,
  'g_13': -0.0,
  'g_14': 1.0,
  'g_15': 1.0,
  'g_16': 1.0,
  'g_17': 1.0,
  'g_18': 1.0,
  'g_19': -0.0,
  'g_20': -0.0,
  'g_21': -0.0,
  'g_22': 1.0,
  'g_23': 1.0,
  'g_24': 1.0,
  'g_25': 1.0,
  'g_26': 1.0,
  'g_27': -0.0

In [140]:
def draw_table(p):
    dict_values = []

    for pf in p.optimal_portfolios:
        dict_values.append(list(pf.values())[:20])
    d2 = dict(zip(p.interaction_effects_dict.keys(), dict_values[1:]))
    d3 = pd.DataFrame(d2)
    d3.columns = [round(x, 2) for x in d3.columns]
    d3 = d3.astype(int)
    d3 = d3.set_index(pd.Index(list(range(1,21))))
    return d3

In [141]:
def draw_table2(p):
    dict_values = []

    for pf in p.optimal_portfolios:
        dict_values.append(list(pf.values())[:20])
    d2 = dict(zip(p.interaction_effects_dict.keys(), dict_values[1:]))
    d3 = pd.DataFrame(d2)
    d3.columns = [round(x, 2) for x in d3.columns]
    d3 = d3.astype(int)
    
    d_list = []

    for col in d3.columns:
        d_list.append(sum(abs(d3[0]-d3[col])))
        
    d3 = d3.append(pd.Series(d_list , index=d3.columns),ignore_index=True)
    d3= d3.set_index(pd.Index(list(df["Action idea name"]) + ["Number of changed decisions"]))
    return d3

In [142]:
draw_table2(s2).to_csv("67_33_50.csv", sep=";") #[0.67, 0.33], 0.5
draw_table2(s1).to_csv("50_50_50.csv", sep=";") #[0.5, 0.5], 0.5
draw_table2(s3).to_csv("100_0_50.csv", sep=";") #[1,0], 0.5
draw_table2(s5).to_csv("67_33_25.csv", sep=";") #[0.67, 0.33], 0.25
draw_table2(s6).to_csv("67_33_75.csv", sep=";") #[0.67, 0.33], 0.75


In [176]:
draw_table2(s6)

,0.00,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,...,0.90,1.00,2.00,3.00,4.00,5.00,6.00,7.00,8.00,9.00
"(1) Promoting the energy efficiency of buildings, the use of waste heat and the use and production of renewable energy",1,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
(2) Enabling decentralized energy production from a security of supply perspective,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
(3) Development and deployment of non-combustion energy technologies and energy storage,1,1,1,1,1,1,1,1,1,1,...,1,1,0,0,0,0,0,0,0,0
"(4) Promoting the production of renewable energy, such as wind, geo-energy and solar energy",1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
(5) Improving energy self-sufficiency to prepare for energy supply disruptions,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
(6) Oil and peat are used only as maintenance security fuels in energy production,1,1,1,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
(7) Improving the climate resilience of arable farming and forests,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
"(8) Ensuring the sustainable production, use and renewal of biomass",1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
(9) Climate management and use of forests. The growth condition of forests is taken care of and the development of forest carbon balances is monitored,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
(10) Promoting the development of manure treatment methods and the use of manure for biogas and fertilizer production,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [ ]:
import matplotlib
matplotlib.rc('text', usetex=True)
matplotlib.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]


In [ ]:
draw_graph(s1)

In [ ]:
df = draw_table(s2)

In [ ]:
df.style.applymap(lambda x: "background-color: green" if x>0 else "background-color: white")

In [ ]:
draw_graph(s2)

In [ ]:
dict_values = []

for pf in s2.optimal_portfolios:
    dict_values.append(list(pf.values())[:20])
d2 = dict(zip(s2.interaction_effects_dict.keys(), dict_values[1:]))

In [ ]:
np.array(d2[0]) - np.array(d2[9])

In [ ]:
def draw_table(p):
    dict_values = []

    for pf in p.optimal_portfolios:
        dict_values.append(list(pf.values())[:20])
    d2 = dict(zip(p.interaction_effects_dict.keys(), dict_values[1:]))
    d3 = pd.DataFrame(d2)
    d3.columns = [round(x, 2) for x in d3.columns]
    d3 = d3.astype(int)
    d3 = d3.set_index(pd.Index(list(range(1,21))))
    return d3

In [ ]:
draw_table(s2)

In [ ]:
def draw_graph(p):
    dict_values = []

    for pf in p.optimal_portfolios:
        dict_values.append(list(pf.values())[:20])
    d2 = dict(zip(p.interaction_effects_dict.keys(), dict_values[1:]))
    
    diff_dict = {}
    for k,v in d2.items():
        diff_dict[k] = np.sum(abs(np.array(d2[0])-np.array(v)))
    plt.plot(diff_dict.keys(), diff_dict.values())
    plt.xlabel(r"$\beta$")
    
    plt.ylabel("")

In [ ]:
draw_graph(s2)

In [ ]:
draw_graph(s1)

In [ ]:
draw_graph(s3)

In [ ]:
p1[0].describe()

In [ ]:
p2[0].describe()

In [ ]:
p3[0].describe()

In [ ]:
draw_graph(s4)

In [ ]:
draw_graph(s5)

In [ ]:
draw_graph(s6)

In [ ]:
dict_values = []

for pf in p.optimal_portfolios:
    dict_values.append(list(pf.values())[:20])

In [ ]:
d2 = dict(zip(p.interaction_effects_dict.keys(), dict_values[1:]))

In [ ]:
d3 = pd.DataFrame(d2)
d3.columns = [round(x, 1) for x in d3.columns]
d3 = d3.astype(int)

In [ ]:
d3.style.applymap(lambda x: "background-color: blue" if x >0 else 'background-color: white')

In [ ]:
diff_dict = {}
for k,v in d2.items():
    diff_dict[k] = np.sum(abs(np.array(d2[0])-np.array(v)))

In [ ]:
plt.plot(diff_dict.keys(), diff_dict.values())
plt.xlabel("$\beta$")
plt.ylabel("Number of different projects selected")

In [ ]:
dict_values = []

for pf in p.optimal_portfolios:
    dict_values.append(list(pf.values())[20:])
    
d2 = dict(zip(p.interaction_effects_dict.keys(), dict_values[1:]))
d3 = pd.DataFrame(d2)
d3.columns = [round(x, 1) for x in d3.columns]
d3 = d3.astype(int)
d3.style.applymap(lambda x: "background-color: blue" if x >0 else 'background-color: white')

In [ ]:
np.sum(abs(np.array(d2[0])-np.array(d2[0.1])))

In [ ]:
len(p.interaction_effects_dict.keys())

In [ ]:
len(p.optimal_portfolios[1:])

In [ ]:
list(p.optimal_portfolios[13].values())[:20]

In [ ]:
np.dot([1,2,3], [0,1,0])

In [ ]:
p.optimal_portfolios[2]

In [ ]:
x = np.dot(list(p.optimal_portfolios[3].values())[:20], p.project_values)
y = np.dot(list(p.optimal_portfolios[3].values())[20:], p.interaction_effects_dict[0.2])
x+y

In [ ]:
p.optimal_values